In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from transformers import BertTokenizer, BertModel, pipeline
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ast
import re
from datetime import datetime

Import data

In [4]:
# Upload the data frame with BERT embeddings:
file_path_embeddings = r'.\..\data\embeddings_output\df_user_embeddings_BERT.csv' # Update this with your file path
df_user_embeddings = pd.read_csv(file_path_embeddings)

In [6]:
df_user_embeddings.head(2)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_758,dim_759,dim_760,dim_761,dim_762,dim_763,dim_764,dim_765,dim_766,dim_767
0,5,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,1,...,-0.094609,-0.411200,-0.062443,-0.214518,0.074292,0.104339,-0.135383,-0.334618,-0.169996,0.315318
1,4,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,1,...,-0.100695,-0.222924,0.112311,-0.042177,0.077755,-0.127271,-0.111048,-0.031468,-0.161275,0.247013


In [7]:
# Upload user DataFrame:
file_path_user_df = r'.\..\data\data_clean\user_clean_data_ecommerce.csv'  # Update this with your file path
df_user = pd.read_csv(file_path_user_df)
df_user.shape

(693941, 12)

In [8]:
missing_values = df_user.isnull().sum()
missing_values

rating                           0
review_title                     0
text_review                      0
user_images                      0
product_ID                       0
parent_ID                        0
user_ID                          0
timestamp                        0
helpful_review_vote              0
user_purchase_verification       0
year                             0
cleaned_text                  1405
dtype: int64

In [9]:
df_user.dropna(inplace=True)
df_user.shape

(692536, 12)

### 1. Normalization of BERT embeddings:

In [10]:
# Select only columns with embedding vectors (e.g., `dim_0`, `dim_1`, ..., `dim_780`)
embedding_columns = [col for col in df_user_embeddings.columns if col.startswith('dim_')]
# Convert to NumPy array
embedding_matrix = df_user_embeddings[embedding_columns].values  
embedding_matrix

array([[ 0.1840019 , -0.02873228,  0.44851863, ..., -0.33461767,
        -0.1699961 ,  0.3153179 ],
       [ 0.08434828,  0.3667166 ,  0.31517023, ..., -0.03146759,
        -0.16127543,  0.24701294],
       [ 0.4264943 ,  0.15444493,  0.34537777, ..., -0.7313684 ,
        -0.3611269 , -0.1057355 ],
       ...,
       [-0.06583165, -0.56118315,  0.17979641, ...,  0.11617286,
         0.07983319, -0.4473238 ],
       [ 0.04332516,  0.05818947,  0.44545922, ..., -0.2437644 ,
         0.11565264, -0.2689667 ],
       [-0.0346208 , -0.08343422,  0.13093697, ..., -0.3016686 ,
        -0.17524104,  0.2820569 ]])

In [11]:
# Compute L2 norm along rows
l2_norms = np.linalg.norm(embedding_matrix, axis=1, keepdims=True)  

In [12]:
# Normalize the embedding columns
normalized_embeddings = embedding_matrix / l2_norms  # Divide each row by its L2 norm
normalized_embeddings

array([[ 0.02091952, -0.00326663,  0.05099292, ..., -0.03804331,
        -0.01932718,  0.03584908],
       [ 0.01000431,  0.04349521,  0.03738144, ..., -0.00373228,
        -0.01912842,  0.0292975 ],
       [ 0.04388997,  0.01589372,  0.03554237, ..., -0.07526416,
        -0.03716309, -0.0108811 ],
       ...,
       [-0.00681894, -0.05812816,  0.01862357, ...,  0.01203335,
         0.00826924, -0.04633445],
       [ 0.00467376,  0.00627726,  0.04805448, ..., -0.02629639,
         0.01247617, -0.02901512],
       [-0.0040209 , -0.00969014,  0.01520717, ..., -0.03503613,
        -0.02035269,  0.0327584 ]])

Generate the output DataFrame with normalized embeddings:

In [13]:
num_dimensions = normalized_embeddings.shape[1]
norm_embedding_columns_names = [f'dim_norm_{i}' for i in range(num_dimensions)]

In [14]:
embeddings_norm = pd.DataFrame(normalized_embeddings, columns=norm_embedding_columns_names)

In [15]:
df_normalized_embeddings = pd.concat([df_user.reset_index(drop=True), embeddings_norm], axis=1)
df_normalized_embeddings.head(2)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_norm_758,dim_norm_759,dim_norm_760,dim_norm_761,dim_norm_762,dim_norm_763,dim_norm_764,dim_norm_765,dim_norm_766,dim_norm_767
0,5,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,1,...,-0.010756,-0.04675,-0.007099,-0.024389,0.008446,0.011863,-0.015392,-0.038043,-0.019327,0.035849
1,4,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,1,...,-0.011943,-0.02644,0.013321,-0.005003,0.009222,-0.015095,-0.013171,-0.003732,-0.019128,0.029297


### 2. Dimentionality reduction with PCA

* Similar steps but here the output is a final user dataframe with vectors of 300 dimensions (In the beginning I have chosen 10 only because that's what they have done in the artile but the variance dropped to 45%, so I set up for now 300 which gives ~95%.) We can always adjust it

In [16]:
pd.set_option('display.max_colwidth', 50)  # Adjust column width
pd.set_option('display.width', 100)       # Adjust width of display

In [17]:
# Perform dimensionality reduction to 300 components using PCA
pca = PCA(n_components=300)

In [18]:
pca_embeddings = pca.fit_transform(normalized_embeddings)

In [19]:
print(pca_embeddings.shape)

(692536, 300)


In [20]:
pca_embeddings

array([[-0.2306334 , -0.01554065, -0.05168869, ..., -0.00320198,
        -0.00405875,  0.00201955],
       [-0.19900984,  0.04691431,  0.01088837, ...,  0.00079175,
        -0.00111065, -0.00182417],
       [ 0.23540958,  0.09816614, -0.09399118, ..., -0.00342424,
        -0.01041647,  0.00149604],
       ...,
       [ 0.22387383, -0.01092586, -0.0832288 , ..., -0.00652219,
        -0.00709183, -0.00869295],
       [-0.11300242,  0.24246496,  0.01130064, ..., -0.00749431,
        -0.00235194, -0.00096008],
       [-0.24762445, -0.11165329, -0.05281798, ..., -0.00555515,
        -0.00427085, -0.00402392]])

### 3. PCA - Variance analysis

* Verify if the size of reduced embeddings is big enough to still keep the meaning of the textual data

In [21]:
embeddings_norm.shape

(692536, 768)

In [22]:
# Fit PCA to the original embeddings
pca = PCA(n_components=300)
pca.fit(embeddings_norm)

# Check explained variance
explained_variance = pca.explained_variance_ratio_
total_variance = sum(explained_variance)

print("Explained Variance Ratio for 300 Dimensions:", explained_variance)
print("Total Explained Variance Captured by 300 Dimensions:", total_variance)

Explained Variance Ratio for 300 Dimensions: [0.1708849  0.04192091 0.03681915 0.03252424 0.02830417 0.02632476
 0.02417032 0.02206136 0.01988072 0.01806828 0.01670958 0.01523421
 0.01472129 0.01379962 0.01262209 0.0117627  0.01051655 0.01030609
 0.00986323 0.00893262 0.00873728 0.00808277 0.00787184 0.00758196
 0.00734143 0.00686471 0.00680506 0.00654111 0.00631994 0.0062196
 0.00607355 0.00583796 0.00537985 0.00525169 0.00512208 0.0050183
 0.00489955 0.0046933  0.00456008 0.00428266 0.0042379  0.00409307
 0.00399598 0.00389995 0.00378321 0.00375769 0.00365085 0.00350971
 0.00345561 0.00339596 0.0033403  0.00330208 0.00322428 0.00317915
 0.00300452 0.00295732 0.00293198 0.00288629 0.00282719 0.0027646
 0.00272828 0.00268521 0.00265742 0.0025835  0.00250174 0.00245645
 0.00238871 0.002326   0.00231586 0.0022487  0.00219941 0.00215871
 0.0021424  0.00211961 0.0020541  0.00204117 0.0020162  0.00198249
 0.00192741 0.00190663 0.00186903 0.00185306 0.00182963 0.00179868
 0.00175412 0.001746

### 4. Generate a merged DataFrame from df_user, including columns for embedding dimensions.

In [23]:
# Create column names for embedding dimensions - normalized data after dimentionality reducion
num_dimensions_pca = pca_embeddings.shape[1]
normalized_columns_PCA = [f'dim_norm_PCA{i}' for i in range(num_dimensions_pca)]

In [24]:
embeddings_df_PCA = pd.DataFrame(pca_embeddings, columns=normalized_columns_PCA)

In [25]:
df_user_normalized_PCA = pd.concat([df_user.reset_index(drop=True), embeddings_df_PCA], axis=1)

In [26]:
df_user_normalized_PCA.head(5)

,rating,review_title,text_review,user_images,product_ID,parent_ID,user_ID,timestamp,helpful_review_vote,user_purchase_verification,...,dim_norm_PCA290,dim_norm_PCA291,dim_norm_PCA292,dim_norm_PCA293,dim_norm_PCA294,dim_norm_PCA295,dim_norm_PCA296,dim_norm_PCA297,dim_norm_PCA298,dim_norm_PCA299
0,5,Such a lovely scent but not overpowering.,This spray is really nice. It smells really go...,[],B00YQ6X8EO,B00YQ6X8EO,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-05 14:08:48.923,0,1,...,-0.001310,0.000084,0.009012,-0.004501,0.001905,0.007937,0.005959,-0.003202,-0.004059,0.002020
1,4,Works great but smells a little weird.,"This product does what I need it to do, I just...",[],B081TJ8YS3,B081TJ8YS3,AGKHLEW2SOWHNMFQIJGBECAF7INQ,2020-05-04 18:10:55.070,1,1,...,-0.007619,-0.012466,0.012960,0.008836,-0.011190,-0.022729,0.000907,0.000792,-0.001111,-0.001824
2,5,Yes!,"Smells good, feels great!",[],B07PNNCSP9,B097R46CSY,AE74DYR3QUGVPZJ3P7RFWBGIX7XQ,2020-05-16 21:41:06.052,2,1,...,0.005019,0.008294,0.010830,0.013650,-0.003329,-0.001900,0.024909,-0.003424,-0.010416,0.001496
3,1,Synthetic feeling,Felt synthetic,[],B09JS339BZ,B09JS339BZ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2022-01-28 18:13:50.220,0,1,...,0.003821,-0.011844,0.004245,-0.009105,0.017510,-0.014226,-0.014142,0.004159,-0.008665,-0.001732
4,5,A+,Love it,[],B08BZ63GMJ,B08BZ63GMJ,AFQLNQNQYFWQZPJQZS6V3NZU4QBQ,2020-12-30 10:02:43.534,0,1,...,0.001345,0.001304,-0.000655,-0.001735,-0.002375,0.001479,-0.002190,-0.002715,-0.004831,-0.001109


Save the output DataFrame:

In [27]:
# Save the DataFrame with normalized & reduced embeddings:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

file_path = rf'.\..\data\embeddings_dim_reduction\df_user_normalized_PCA{timestamp}.csv'
df_user_normalized_PCA.to_csv(file_path, index=False)